In [ ]:
#convert HDF5 to tif

In [ ]:
def generateHDF_from_FastZ2Color(output, path, fastz_tif):
    #output = output path for .hdf5 file
    #path = directory where tif images are stored
    #fastz_tif = list of tif_images in path
    rgb = getMeanInTime_from_TIF(path, fastz_tif)
    #plt.imshow(np.max(rgb, axis = 0))disco
    rgb2 =np.zeros([1, rgb.shape[0], rgb.shape[1], rgb.shape[2], rgb.shape[3]], dtype = rgb.dtype)
        
    #for layer in green_range
    rgb2[0, :, :, :, 0] = rgb[:, :, :,0]
    rgb2[0, :, :, :, 1] = rgb[:, :, :,1]
    
    rgb2[rgb2 > 1] = 1
    rgb2 = np.rollaxis(rgb2, 3, 2)
    rgb2 = skimage.img_as_int(rgb2)
    hdf5_file = tables.open_file(output, mode='w')
    #filters = tables.Filters(complevel=5, complib='blosc') #faster but poor compression
    filters = tables.Filters(complevel=3, complib='zlib') #slower but better compression
    data_storage = hdf5_file.create_carray(hdf5_file.root, 'data',
                                  tables.Atom.from_dtype(rgb2.dtype),
                                  shape = rgb2.shape,
                                  filters=filters)
    data_storage[:] = rgb2
    data_storage.flush()
    hdf5_file.close()
    for file1 in fastz_tif:
        target = Path(path) / file1
        target.unlink()
    #remove the matfile
    for file in Path(path).glob('*.mat'):
        file.unlink()

In [ ]:
def generateMIPfromHDFcolor(path):
    #path = ///.2color_z_#####.HDF5 or from .tif file
    if '.hdf5' in path:
        HDF5_file = tables.open_file(path, mode='r')
        color = HDF5_file.root.data[:, : , :,:]
        color = np.transpose(np.squeeze(color), [0, 2,1, 3])
    else:
        color = generate2ColorArray(path)
    color = np.squeeze(color)
    color = color.max(axis = 0)
    color = skimage.img_as_ubyte(color)
    return color